###  损失函数：交叉熵    
交叉熵用来衡量两个概率分布之间“差异”的数学指标  

- 真实分布（Target）：模型应该预测的词  
- 预测分布（Prediction）: 模型认为词可能的分布  
- 交叉熵：预测的分布离准确的词越远，交叉熵的值越大；反之，如果预测出了正确的词，则损失接近于0 

#### 1. 数学表示  
对于真实分布p和预测分布q，交叉熵的公式为：  
$$H(p, q) = - \sum_{x \in X} p(x) \log q(x)$$


在大模型任务中，正确答案通常是确定的，公式会简化为
$$ Loss=-log(p_correct) $$
即模型对正确的词预测的概率取对数加负号

### 2. 技术细节  
大模型的训练本质是一个多分类任务。每生成一个词，模型都要从词表中选出正确的词 
- 逐词元计算：模型在训练阶段，会根据当前给定的前一个i-1个词元，预测第i个词元出现的概率分布  
- 损失累加：将整个序列（长度为N）中每个位置计算出的交叉熵损失取平均值，得到整个句子的总损失  
- Softmax: 计算交叉熵之前，模型输出的是原始分数。必须经过Softmax，将这些分数转化为总和为1的概率分布 

#### 3.Z-loss与稳定性
- 上溢（overflow）：单精度浮点数（FP32）能表示的最大值约为3.4e+38，超过这个值的数会被截断为inf，导致计算错误
- 下溢（underflow）：FP32能表示的最小值约为1.2e-38，低于这个值的数会被截断为0，导致计算错误

- 对策：减去最大值M 
利用恒等式：
$$log(\sum_{i=1}^ne^x_i) = M + log(\sum_{i=1}^ne^{x_i-M})$$

其中M=max(0)  
  - 减去M后，$o_j$-M的最大值为0  
  - exp(0)=1 ，保证了求和项中至少有一个为1，彻底杜绝了分母为0的下溢风险  
  - 所有指数都在（0，1]之间，因此对数不会出现负数，杜绝了上溢风险


In [3]:
import torch
import torch.nn.functional as F 

# shape: (batch_size, num_classes] = [3,5]
logits = torch.tensor([
    [1.2, 3.1, 0.7, 2.4, 0.9],  # 第1个样本
    [0.5, 1.8, 2.9, 1.1, 0.3],  # 第2个样本
    [4.2, 2.5, 3.7, 1.9, 0.8]   # 第3个样本
], dtype=torch.float32)


batch_size = logits.shape[0]
num_classes = logits.shape[1]
#目标类别: shape :[batch_size]=[3]
targets = torch.tensor([1, 2, 0], dtype=torch.long)

print(logits.shape)
print(targets.shape)
print(logits)
print(targets)


torch.Size([3, 5])
torch.Size([3])
tensor([[1.2000, 3.1000, 0.7000, 2.4000, 0.9000],
        [0.5000, 1.8000, 2.9000, 1.1000, 0.3000],
        [4.2000, 2.5000, 3.7000, 1.9000, 0.8000]])
tensor([1, 2, 0])


In [4]:
#计算最大值M
M = torch.max(logits,dim=1)[0]
print(M)

#将M扩展为和logits相同的形状，方便后续广播相减
M_expand = M.unsqueeze(1).expand(logits.shape)
print(M_expand.shape)
print(M_expand)

tensor([3.1000, 2.9000, 4.2000])
torch.Size([3, 5])
tensor([[3.1000, 3.1000, 3.1000, 3.1000, 3.1000],
        [2.9000, 2.9000, 2.9000, 2.9000, 2.9000],
        [4.2000, 4.2000, 4.2000, 4.2000, 4.2000]])


In [6]:
#提取目标位置的分值
#logits - M
logits_adjusted = logits - M_expand
print(logits_adjusted)

#提取每个样本对应目标类别的分值
targets_scores = logits_adjusted[range(batch_size),targets]
print(targets_scores)

tensor([[-1.9000,  0.0000, -2.4000, -0.7000, -2.2000],
        [-2.4000, -1.1000,  0.0000, -1.8000, -2.6000],
        [ 0.0000, -1.7000, -0.5000, -2.3000, -3.4000]])
tensor([0., 0., 0.])


In [7]:
#计算log_sum_exp
exp_logits = torch.exp(logits_adjusted)
print(exp_logits)

#计算每行的exp_logits之和
sum_exp_logits = torch.sum(exp_logits, dim=1)
print(sum_exp_logits)

#计算对数和
log_sum_exp = torch.log(sum_exp_logits)
print(log_sum_exp)

tensor([[0.1496, 1.0000, 0.0907, 0.4966, 0.1108],
        [0.0907, 0.3329, 1.0000, 0.1653, 0.0743],
        [1.0000, 0.1827, 0.6065, 0.1003, 0.0334]])
tensor([1.8477, 1.6632, 1.9228])
tensor([0.6139, 0.5087, 0.6538])


In [8]:
#计算每个token的loss
Token_loss = -targets_scores + log_sum_exp
print(Token_loss)


tensor([0.6139, 0.5087, 0.6538])


In [9]:
#求全批次
avg_loss = torch.mean(Token_loss)
print(avg_loss)

tensor(0.5922)


In [10]:
#验证
loss = F.cross_entropy(logits, targets)
print(loss)

tensor(0.5922)
